In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
def setup_download_from_s3( rel_fp, recursive=True ):
    s3_fp = 's3://mousebrainatlas-data/'+rel_fp
    local_fp = os.environ['ROOT_DIR']+rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return 
    
    if recursive:
        ! aws s3 cp --recursive $s3_fp $local_fp
    else:
        ! aws s3 cp $s3_fp $local_fp

In [3]:
setup_download_from_s3('CSHL_new_regions/MD589/')

ALREADY DOWNLOADED FILE


In [2]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [10]:
stack = 'MD589'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
patch_file = 'CSHL_patches_200'
scripts_dir = os.environ['REPO_DIR']

In [5]:
Structure4.drop()

`kui_diffusionmap`.`structure4` (27 tuples)
Proceed? [yes, No]: No


In [6]:
@schema
class Structure5(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    positive_s3_fp  : varchar(200)  # (str) file path
    negative_s3_fp  : varchar(200)  # (str) file path
    """

In [11]:
paired_structures = ['5N', '6N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC'] #, '7N'
all_structures = paired_structures + singular_structures
stack = 'MD589'
s3_fp = patch_file+'_features/'+stack+'/' #s3://mousebrainatlas-data/
print(s3_fp)

CSHL_patches_200_features/MD589/


In [8]:
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure5.insert1(dict(id=n,
                           structure=struc,
                           positive_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_positive.pkl',
                           negative_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_negative.pkl',)
                      ,skip_duplicates=False)
    n+=1


Adding 5N to the database

Adding 6N to the database

Adding 7n to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [9]:
structureTable = Structure5.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,id,structure,positive_s3_fp,negative_s3_fp
0,0,5N,CSHL_patches_200_features/MD589/5N/MD589_5N_po...,CSHL_patches_200_features/MD589/5N/MD589_5N_ne...
1,1,6N,CSHL_patches_200_features/MD589/6N/MD589_6N_po...,CSHL_patches_200_features/MD589/6N/MD589_6N_ne...
2,2,7n,CSHL_patches_200_features/MD589/7n/MD589_7n_po...,CSHL_patches_200_features/MD589/7n/MD589_7n_ne...
3,3,Amb,CSHL_patches_200_features/MD589/Amb/MD589_Amb_...,CSHL_patches_200_features/MD589/Amb/MD589_Amb_...
4,4,LC,CSHL_patches_200_features/MD589/LC/MD589_LC_po...,CSHL_patches_200_features/MD589/LC/MD589_LC_ne...
5,5,LRt,CSHL_patches_200_features/MD589/LRt/MD589_LRt_...,CSHL_patches_200_features/MD589/LRt/MD589_LRt_...
6,6,Pn,CSHL_patches_200_features/MD589/Pn/MD589_Pn_po...,CSHL_patches_200_features/MD589/Pn/MD589_Pn_ne...
7,7,Tz,CSHL_patches_200_features/MD589/Tz/MD589_Tz_po...,CSHL_patches_200_features/MD589/Tz/MD589_Tz_ne...
8,8,VLL,CSHL_patches_200_features/MD589/VLL/MD589_VLL_...,CSHL_patches_200_features/MD589/VLL/MD589_VLL_...
9,9,RMC,CSHL_patches_200_features/MD589/RMC/MD589_RMC_...,CSHL_patches_200_features/MD589/RMC/MD589_RMC_...


In [14]:
Process5.drop()

`kui_diffusionmap`.`__process5` (0 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [15]:
@schema
class Process5(dj.Computed):
    definition="""
    -> Structure2
    -----
    size_positive_properties : int   #size of file
    size_negative_properties : int   #size of file
    """

    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure5 & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            key_item = 'size_'+state+'_properties'
            s3_fp = (Structure5 & key).fetch1(item_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file $patch_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                pkl_out_file = os.environ['ROOT_DIR']+s3_fp
                ! aws s3 cp $pkl_out_file $upload_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [16]:
Process5.populate()

populating for  5N
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-25 18:32:26,890	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_18-32-26_91649/logs.
2019-06-25 18:32:27,001	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:41297 to respond...
2019-06-25 18:32:27,129	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:37626 to respond...
2019-06-25 18:32:27,132	INFO services.py:804 -- Starting Redis shard with 1.72 GB max memory.
2019-06-25 18:32:27,143	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_18-32-26_91649/logs.
2019-06-25 18:32:27,144	INFO services.py:1427 -- Starting the Plasma object store with 2.58 GB memory using /tmp.
/U

populating for  7n
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-25 18:37:30,335	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_18-37-30_91758/logs.
2019-06-25 18:37:30,440	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:44553 to respond...
2019-06-25 18:37:30,559	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:42308 to respond...
2019-06-25 18:37:30,562	INFO services.py:804 -- Starting Redis shard with 1.72 GB max memory.
2019-06-25 18:37:30,573	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_18-37-30_91758/logs.
2019-06-25 18:37:30,574	INFO services.py:1427 -- Starting the Plasma object store with 2.58 GB memory using /tmp.
/U

(pid=91807) 7n_negative 1 0 / 1000
(pid=91807) 7n_negative 11 10 / 1000
(pid=91807) 7n_negative 21 20 / 1000
(pid=91807) 7n_negative 31 30 / 1000
(pid=91807) 7n_negative 41 40 / 1000
(pid=91807) 7n_negative 51 50 / 1000
(pid=91807) 7n_negative 61 60 / 1000
(pid=91807) 7n_negative 71 70 / 1000
(pid=91807) 7n_negative 81 80 / 1000
(pid=91807) 7n_negative 91 90 / 1000
(pid=91807) 7n_negative 101 100 / 1000
(pid=91807) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD589/7n_surround_500um_noclass/158_1.tif std= 8.741377053028643 too blank, skipping
(pid=91807) 7n_negative 110 110 / 1000
(pid=91807) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD589/7n_surround_500um_noclass/154_5.tif std= 7.305912083699932 too blank, skipping
(pid=91807) 7n_negative 119 120 / 1000
(pid=91807) 7n_negative 129 130 / 1000
(pid=91807) 7n_negative 139 140 / 1000
(pid=91807) 7n_negative 149 150 / 1000
(pid=91807) 7n_negative 159 160 / 1000
(pid=91807) 7n_negative 1

(pid=91858) Amb_positive 126
(pid=91858) Amb_positive finished in  69.6 seconds
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD589/Amb/MD589_Amb_positive.pkl to s3://mousebrainatlas-data/CSHL_patches_200_features/MD589/Amb/MD589_Amb_positive.pkl
/Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD589/Amb/MD589_Amb_positive.pkl s3://mousebrainatlas-data/CSHL_patches_200_features/MD589/Amb/MD589_Amb_positive.pkl
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-25 18:55:18,672	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_18-55-18_91876/logs.
2019-06-25 18:55:18,779	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:15036 to respond...
2019-06

(pid=91965) LC_negative 181 180 / 240
(pid=91965) LC_negative 191 190 / 240
(pid=91965) LC_negative 201 200 / 240
(pid=91965) LC_negative 211 210 / 240
(pid=91965) LC_negative 221 220 / 240
(pid=91965) LC_negative 231 230 / 240
(pid=91965) LC_negative 240
(pid=91965) LC_negative finished in 129.5 seconds
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD589/LC/MD589_LC_negative.pkl to s3://mousebrainatlas-data/CSHL_patches_200_features/MD589/LC/MD589_LC_negative.pkl
/Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD589/LC/MD589_LC_negative.pkl s3://mousebrainatlas-data/CSHL_patches_200_features/MD589/LC/MD589_LC_negative.pkl
populating for  LRt
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-25 19:01:

...

(pid=93517) SC_positive 151 150 / 1000
(pid=93517) SC_positive 161 160 / 1000
(pid=93517) SC_positive 171 170 / 1000
(pid=93517) SC_positive 181 180 / 1000
(pid=93517) SC_positive 191 190 / 1000
(pid=93517) SC_positive 201 200 / 1000
(pid=93517) SC_positive 211 210 / 1000
(pid=93517) SC_positive 221 220 / 1000
(pid=93517) SC_positive 231 230 / 1000
(pid=93517) SC_positive 241 240 / 1000
(pid=93517) SC_positive 251 250 / 1000
(pid=93517) SC_positive 261 260 / 1000
(pid=93517) SC_positive 271 270 / 1000
(pid=93517) SC_positive 281 280 / 1000
(pid=93517) SC_positive 291 290 / 1000
(pid=93517) SC_positive 301 300 / 1000
(pid=93517) SC_positive 311 310 / 1000
(pid=93517) SC_positive 321 320 / 1000
(pid=93517) SC_positive 331 330 / 1000
(pid=93517) SC_positive 341 340 / 1000
(pid=93517) SC_positive 351 350 / 1000
(pid=93517) SC_positive 361 360 / 1000
(pid=93517) SC_positive 371 370 / 1000
(pid=93517) SC_positive 381 380 / 1000
(pid=93517) SC_positive 391 390 / 1000
(pid=93517) SC_positive 4

(pid=93562) SC_negative 329 330 / 1000
(pid=93562) SC_negative 339 340 / 1000
(pid=93562) SC_negative 349 350 / 1000
(pid=93562) SC_negative 359 360 / 1000
(pid=93562) SC_negative 369 370 / 1000
(pid=93562) SC_negative 379 380 / 1000
(pid=93562) SC_negative 389 390 / 1000
(pid=93562) SC_negative 399 400 / 1000
(pid=93562) SC_negative 409 410 / 1000
(pid=93562) SC_negative 419 420 / 1000
(pid=93562) SC_negative 429 430 / 1000
(pid=93562) SC_negative 439 440 / 1000
(pid=93562) SC_negative 449 450 / 1000
(pid=93562) SC_negative 459 460 / 1000
(pid=93562) SC_negative 469 470 / 1000
(pid=93562) SC_negative 479 480 / 1000
(pid=93562) SC_negative 489 490 / 1000
(pid=93562) SC_negative 499 500 / 1000
(pid=93562) SC_negative 509 510 / 1000
(pid=93562) SC_negative 519 520 / 1000
(pid=93562) SC_negative 529 530 / 1000
(pid=93562) SC_negative 539 540 / 1000
(pid=93562) SC_negative 549 550 / 1000
(pid=93562) SC_negative 559 560 / 1000
(pid=93562) SC_negative 569 570 / 1000
(pid=93562) SC_negative 5

(pid=93604) IC_positive 491 490 / 1000
(pid=93604) IC_positive 501 500 / 1000
(pid=93604) IC_positive 511 510 / 1000
(pid=93604) IC_positive 521 520 / 1000
(pid=93604) IC_positive 531 530 / 1000
(pid=93604) IC_positive 541 540 / 1000
(pid=93604) IC_positive 551 550 / 1000
(pid=93604) IC_positive 561 560 / 1000
(pid=93604) IC_positive 571 570 / 1000
(pid=93604) IC_positive 581 580 / 1000
(pid=93604) IC_positive 591 590 / 1000
(pid=93604) IC_positive 601 600 / 1000
(pid=93604) IC_positive 611 610 / 1000
(pid=93604) IC_positive 621 620 / 1000
(pid=93604) IC_positive 631 630 / 1000
(pid=93604) IC_positive 641 640 / 1000
(pid=93604) IC_positive 651 650 / 1000
(pid=93604) IC_positive 661 660 / 1000
(pid=93604) IC_positive 671 670 / 1000
(pid=93604) IC_positive 681 680 / 1000
(pid=93604) IC_positive 691 690 / 1000
(pid=93604) IC_positive 701 700 / 1000
(pid=93604) IC_positive 711 710 / 1000
(pid=93604) IC_positive 721 720 / 1000
(pid=93604) IC_positive 731 730 / 1000
(pid=93604) IC_positive 7

(pid=93644) IC_negative 690 690 / 1000
(pid=93644) IC_negative 700 700 / 1000
(pid=93644) IC_negative 710 710 / 1000
(pid=93644) IC_negative 720 720 / 1000
(pid=93644) IC_negative 730 730 / 1000
(pid=93644) IC_negative 740 740 / 1000
(pid=93644) IC_negative 750 750 / 1000
(pid=93644) IC_negative 760 760 / 1000
(pid=93644) IC_negative 770 770 / 1000
(pid=93644) IC_negative 780 780 / 1000
(pid=93644) IC_negative 790 790 / 1000
(pid=93644) IC_negative 800 800 / 1000
(pid=93644) IC_negative 810 810 / 1000
(pid=93644) IC_negative 820 820 / 1000
(pid=93644) IC_negative 830 830 / 1000
(pid=93644) IC_negative 840 840 / 1000
(pid=93644) IC_negative 850 850 / 1000
(pid=93644) IC_negative 860 860 / 1000
(pid=93644) IC_negative 870 870 / 1000
(pid=93644) IC_negative 880 880 / 1000
(pid=93644) IC_negative 890 890 / 1000
(pid=93644) IC_negative 900 900 / 1000
(pid=93644) IC_negative 910 910 / 1000
(pid=93644) IC_negative 920 920 / 1000
(pid=93644) IC_negative 930 930 / 1000
(pid=93644) IC_negative 9

DataJointError: fetch1 should only return one tuple. 0 tuples were found